### TF Differentiation of Ipsen upper bound formula

Testing numerically: Is the matrix-vector multiplication norm a convex function? Or a concave function? Or indefinite?

The problem seems to be concave (in conic optimization terms, which needs to be a minimization problem). We cannot define a convex set that bounds our desired objective value. Generally the zero-one problem was NP-hard, but now we know that a quadratic, continuous approximation may also be NP-hard (and the shape of the convex cone would push us to the extrema towards nearly integer solutions, so there is not an easy and efficient way to solve it).

In [2]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [3]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(np.float64)
# Only take 10 columns for testing.
np.random.seed(4)
def sample_hermitian(mat, choices):
    return mat[:, choices][choices, :]
data = sample_hermitian(data, np.random.choice(data.shape[0], 10))
# Synthesize columns holding observations, reproducing the cov matrix.
raw_data = np.linalg.cholesky(data).T
np.testing.assert_allclose(data, raw_data.T @ raw_data)
data

array([[ 1.    , -0.3093, -0.593 ,  0.2698,  0.0679,  0.349 , -0.7985, -0.1988, -0.0798, -0.0217],
       [-0.3093,  1.    ,  0.3678,  0.1036,  0.0957, -0.0789,  0.3371, -0.2643,  0.485 , -0.384 ],
       [-0.593 ,  0.3678,  1.    , -0.3317,  0.102 , -0.3785,  0.4811,  0.2561,  0.3855, -0.1261],
       [ 0.2698,  0.1036, -0.3317,  1.    ,  0.248 ,  0.2302,  0.0207, -0.6127,  0.2957, -0.1031],
       [ 0.0679,  0.0957,  0.102 ,  0.248 ,  1.    , -0.0462,  0.0432, -0.215 ,  0.495 , -0.1981],
       [ 0.349 , -0.0789, -0.3785,  0.2302, -0.0462,  1.    , -0.2407, -0.278 ,  0.0893,  0.156 ],
       [-0.7985,  0.3371,  0.4811,  0.0207,  0.0432, -0.2407,  1.    ,  0.0096,  0.2185,  0.077 ],
       [-0.1988, -0.2643,  0.2561, -0.6127, -0.215 , -0.278 ,  0.0096,  1.    , -0.2101,  0.373 ],
       [-0.0798,  0.485 ,  0.3855,  0.2957,  0.495 ,  0.0893,  0.2185, -0.2101,  1.    , -0.1276],
       [-0.0217, -0.384 , -0.1261, -0.1031, -0.1981,  0.156 ,  0.077 ,  0.373 , -0.1276,  1.    ]])

In [4]:
selected = np.asarray([0, 1])
U = np.linalg.svd(raw_data[:, selected], full_matrices=0)[0][:, 0]
XU = raw_data.T @ U
sigma_residual = data - (XU @ XU.T)

In [5]:
import tensorflow as tf

2023-02-07 09:08:31.917641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 09:08:32.693717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-07 09:08:32.693752: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-07 09:08:32.799650: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-07 09:08:34.405291: W tensorflow/stream_executor/platform/de

In [22]:
@tf.function
def ipsen_nonlinear(sigma_residual, v, S):
    # Numerator: Use S (ideally symmetric, rank-one, {0,1}^{n,n}) to shield
    # the entries in S that ought to be removed rows/columns (Hadamard product).
    # S with a support of the same k rows/columns can be multiplied directly
    # against v.
    # Denominator: Validate this further. Maybe there is something fishy going
    # on so that the Hessian is almost PSD, but not quite.
    return tf.linalg.norm(
        tf.linalg.matmul(sigma_residual * S, v[:, None])
    ) / tf.math.sqrt(
        tf.math.reduce_sum(S * v * tf.transpose(v))
    )

In [23]:
ipsen_nonlinear(tf.constant(sigma_residual, tf.float64), tf.constant(XU, tf.float64), tf.cast(tf.eye(10), tf.float64))

<tf.Tensor: shape=(), dtype=float64, numpy=1.4086621193212558>

In [24]:
@tf.function
def ipsen_nonlinear_hessian(sigma_residual, v, S):
    return tf.hessians(ipsen_nonlinear(sigma_residual, v, S), S)

In [25]:
select_v = np.asarray([1., 1., 0., 0., 0., 1., 0., 1., 0., 0.,])[:, None]
S = select_v @ select_v.T

select_v = np.random.uniform(size=[10, 1])

S = select_v @ select_v.T

In [26]:
h = ipsen_nonlinear_hessian(tf.constant(sigma_residual, tf.float64), tf.constant(XU, tf.float64), tf.constant(S, tf.float64))[0].numpy()
h

array([[[[ 0.2847, -0.5058, -0.4139, ...,  0.025 , -0.2029,  0.1299],
         [ 0.063 , -0.0101, -0.0185, ...,  0.0024, -0.0119,  0.0148],
         [ 0.0149,  0.0149,  0.008 , ...,  0.    ,  0.0028,  0.0011],
         ...,
         [ 0.0329, -0.0037, -0.0029, ...,  0.0005, -0.0051,  0.005 ],
         [ 0.0149,  0.0149,  0.008 , ...,  0.    ,  0.0028,  0.0011],
         [ 0.0149,  0.0149,  0.008 , ...,  0.    ,  0.0028,  0.0011]],

        [[-0.5058,  0.9691,  0.786 , ..., -0.0466,  0.3832, -0.2405],
         [-0.2429,  0.1224,  0.1313, ..., -0.0118,  0.073 , -0.0686],
         [-0.0023, -0.0023, -0.0012, ..., -0.    , -0.0004, -0.0002],
         ...,
         [-0.0927,  0.0903,  0.0535, ..., -0.0024,  0.0387, -0.0197],
         [-0.0023, -0.0023, -0.0012, ..., -0.    , -0.0004, -0.0002],
         [-0.0023, -0.0023, -0.0012, ..., -0.    , -0.0004, -0.0002]],

        [[-0.4139,  0.786 ,  0.6382, ..., -0.0379,  0.3114, -0.1959],
         [-0.1856,  0.0896,  0.0976, ..., -0.0089,  0.0545

In [27]:
# Hessian is not positive semidefinite nor negative semidefinite.
np.linalg.eigvalsh(h.reshape([100, 100]))

array([-0.3239, -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,
       -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,
       -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.5869,  2.3939,  2.6733,  2.7571,  2.7949,  2.8092,  3.0007,  3.1019,  3.2336,  3.3379])